# Recipe_Ingr

An early attempt to train a network to generate the ingredient list of a recipe given its title, as the first step in hierarchical generation of a full recipe.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pathlib
import os
import pickle

import re
import spacy

from collections import namedtuple
from tqdm import tqdm

In [ ]:
CACHE_DIR = './drive/Shared drives/Capstone/tmp'
pathlib.Path(CACHE_DIR).mkdir(exist_ok=True)
dataset_path = os.path.join(CACHE_DIR, 'emoji_text_recipes.pkl')

In [ ]:
if not os.path.exists(dataset_path):
    raise SystemExit("Run preprocess_rnn_word.ipynb to generate data file before continuing")
else:
    recipes = pd.read_pickle(dataset_path)

In [ ]:
recipes

0         🍴 Slow Cooker Chicken and Dumplings\n\n🥑\n• 4 ...
1         🍴 Awesome Slow Cooker Pot Roast\n\n🥑\n• 2 (10....
2         🍴 Brown Sugar Meatloaf\n\n🥑\n• 1/2 cup packed ...
3         🍴 Best Chocolate Chip Cookies\n\n🥑\n• 1 cup bu...
4         🍴 Homemade Mac and Cheese Casserole\n\n🥑\n• 8 ...
                                ...                        
125158    🍴 Cream Horns\n\n🥑\n• 1 sheet frozen puff past...
125159    🍴 Summer Corn Salad\n\n🥑\n• 4 ears fresh corn\...
125160    🍴 Zucchini Stuffed Tomatoes\n\n🥑\n• 4 large pl...
125162    🍴 Chocolate Cake with Armagnac Ice Cream\n\n🥑\...
125163    🍴 Crabby Bisque\n\n🥑\n• 3 (10.5-ounce) cans re...
Length: 105789, dtype: object

In [ ]:
# Makes the dataset small, or at least tractable
recipes = recipes[:20000]

In [ ]:
class IngredientProcessor(object):
  def __init__(self, use_cache=True):
    self.MEASURES = set(["tbsp", "tablespoon", "tablespoons",
            "tsp", "teaspoon", "teaspoons",
            "fl", "oz", "ounce", "ounces",
            "lb", "pound", "pounds",
            "cm", "centimeter", "centimeters", "centimetre", "centimetres",
            "inch", "inches",
            "can", "cans",
            "cup", "cups",
            "pkg", "package", "packages",
            "piece", "pieces",
            "slice", "slices",
            "small", "medium", "large",
            "sliced", "diced", "minced", "chopped",
            "extra"])
    self.CONTEXT_NEEDERS = set(["extract", "root", "sauce", "cream",
            "broth", "soup", "soda", "oil", "puree", "powder",
            "mix", "roast", "paste",
            "chip", "chips",
            "bean", "beans",
            "pepper", "peppers"])
    self.nlp = spacy.load("en_core_web_sm", disable=["textcat", "ner", "entity_ruler", "sentencizer", "merge_noun_chunks", "merge_entities", "merge_subtokens"])
    self.use_cache = use_cache
    self.cache = {}

  def deduce_core_ingredient(self, ingr_phrase):
    '''
      ingr_phrase: str
      returns: str

      Attempts to deduce a "canonical form" of the ingredient
      contained in ingr_phrase.
    '''
    words_in_phrase = ingr_phrase.split(" ")
    ingredWord = ""
    if len(words_in_phrase) >= 2 and words_in_phrase[-1].lower() in self.CONTEXT_NEEDERS:
      ingredWord = " ".join(words_in_phrase[-2:])
      # For diagnostic purposes, tell if this has happened?
      # print("DCG: ", words_in_phrase, ingredWord)
    else:
      ingredWord = words_in_phrase[-1]
    return ingredWord
  
  def cleanup_ingredient(self, ingredient):
    '''
      ingredient: str
      returns: List[str]

      Cleans up ingredient string and returns a list of ingredients
      in canonical form.
    '''
    # First clean up by removing unnecessary information
    
    # Uses regex to remove parenthesised portions and numbers
    # https://www.kite.com/python/answers/how-to-use-regular-expressions-to-remove-text-within-parentheses-in-python
    ingredient = re.sub(r"\([^()]*\)|[0-9]|/|\.|,", "", ingredient)

    # Filter out measure words
    
    ingredient = " ".join(word for word in ingredient.split() if not word.lower() in self.MEASURES)
    
    ingredient = ingredient.strip()

    if self.use_cache:
      if ingredient in self.cache:
        return self.cache[ingredient]

    # Apply spacy to get the most important noun phrase (hopefully)
    ingr_doc = self.nlp(ingredient)
    noun_phrases = [chunk.text for chunk in ingr_doc.noun_chunks]

    # Try to deduce the ingredient
    num_phrases = len(noun_phrases)
    ingr_list = ""
    if num_phrases == 0:
      ingr_list = [ingredient]
    else:
      ingr_list = [self.deduce_core_ingredient(phrase) for phrase in noun_phrases]

    if self.use_cache:
      self.cache[ingredient] = ingr_list

    return ingr_list

# Process the recipes to get the ingredients!

And other operations necessary.
Note: If you have already done this just unpickle the saved results.

In [ ]:
def flatten(lists):
  return [item for sublist in lists for item in sublist]

ProcessedRecipe = namedtuple('ProcessedRecipe', ['title', 'ingredients', 'instructions'])

IngrProc = IngredientProcessor()

def process_recipe(recipe):
  _, title, ingredients, instructions = re.split("🍴|🥑|🥣", recipe)

  # Process title
  title = title.strip()

  # Process ingredients
  ingredients = ingredients.replace('\n', '').split("•")
  ingredients = [ingredient.strip() for ingredient in ingredients if len(ingredient) > 0]
  cleaned_up_ingrs = flatten([IngrProc.cleanup_ingredient(ingredient) for ingredient in ingredients])

  # Process instructions
  instructions = instructions.replace('\n', '').split("‣")
  cleaned_up_instrs = [instruction.strip() for instruction in instructions if len(instruction) > 0]

  return ProcessedRecipe(title, cleaned_up_ingrs, cleaned_up_instrs)

def extract_title(recipe):
  _, title, _, _ = re.split("🍴|🥑|🥣", recipe)
  title = title.strip()
  return title

def extract_cleaned_ingredients(recipe):
  _, _, ingredients, _ = re.split("🍴|🥑|🥣", recipe)
  ingredients = ingredients.replace('\n', '').split("•")
  ingredients = [ingredient.strip() for ingredient in ingredients if len(ingredient) > 0]
  cleaned_up_ingrs = flatten([IngrProc.cleanup_ingredient(ingredient) for ingredient in ingredients]) 
  return cleaned_up_ingrs

def extract_cleaned_instructions(recipe):
  _, _, _, instructions = re.split("🍴|🥑|🥣", recipe)
  instructions = instructions.replace('\n', '').split("‣")
  cleaned_up_instrs = [instruction.strip() for instruction in instructions if len(instruction) > 0]
  return cleaned_up_instrs 

In [ ]:
ingr_set2 = set()

for recipe in tqdm(recipes):
  ingrs = extract_cleaned_ingredients(recipe)
  for ingr in ingrs:
    ingr_set2.add(ingr)

len(ingr_set2)

In [ ]:
ingr_set3 = set()

for recipe in tqdm(recipes[:10]):
  ingrs = extract_cleaned_ingredients(recipe)
  for ingr in ingrs:
    ingr_set3.add(ingr)

len(ingr_set3)

100%|██████████| 10/10 [00:06<00:00,  1.47it/s]


53

In [ ]:
IngrProc.cache

{}

In [ ]:
IngrProc.cache

{}

In [ ]:
from tqdm import tqdm

ingr_set = set()
for recipe in tqdm(recipes):
  title, ingrs, instrs = process_recipe(recipe)
  for ingr in ingrs:
    ingr_set.add(ingr)

len(ingr_set)

100%|██████████| 100/100 [02:02<00:00,  1.22s/it]


215

In [ ]:
ingr_set

## Now actually process all the recipes

Or at least the first 20000

In [ ]:
df_recipes = pd.DataFrame({'recipe': recipes})

In [ ]:
len(df_recipes)

20000

In [ ]:
# Process all of the recipes.

tqdm.pandas()

print("-- Extracting titles")
df_recipes['title'] = df_recipes.progress_apply(lambda row: extract_title(row['recipe']), axis=1)
print("-- Extracting and processing ingredients")
df_recipes['ingredients'] = df_recipes.progress_apply(lambda row: extract_cleaned_ingredients(row['recipe']), axis=1)
print("-- Extracting instructions")
df_recipes['instructions'] = df_recipes.progress_apply(lambda row: extract_cleaned_instructions(row['recipe']), axis=1)

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel





  0%|          | 0/20000 [00:00<?, ?it/s]




 19%|█▉        | 3778/20000 [00:00<00:00, 37777.15it/s]

-- Extracting titles







 36%|███▋      | 7282/20000 [00:00<00:00, 36909.42it/s]




 57%|█████▋    | 11385/20000 [00:00<00:00, 38054.87it/s]




 75%|███████▌  | 15049/20000 [00:00<00:00, 37617.31it/s]




100%|██████████| 20000/20000 [00:00<00:00, 35251.38it/s]





  0%|          | 0/20000 [00:00<?, ?it/s]

-- Extracting and processing ingredients


Streaming output truncated to the last 5000 lines.




 88%|████████▊ | 17691/20000 [45:28<07:16,  5.29it/s]




 88%|████████▊ | 17692/20000 [45:28<08:53,  4.32it/s]




 88%|████████▊ | 17694/20000 [45:29<08:09,  4.71it/s]




 88%|████████▊ | 17695/20000 [45:29<11:34,  3.32it/s]




 88%|████████▊ | 17696/20000 [45:29<12:07,  3.17it/s]




 88%|████████▊ | 17697/20000 [45:30<12:32,  3.06it/s]




 88%|████████▊ | 17698/20000 [45:30<16:15,  2.36it/s]




 88%|████████▊ | 17699/20000 [45:31<17:37,  2.18it/s]




 88%|████████▊ | 17700/20000 [45:31<14:18,  2.68it/s]




 89%|████████▊ | 17703/20000 [45:31<10:42,  3.58it/s]




 89%|████████▊ | 17704/20000 [45:31<09:34,  4.00it/s]




 89%|████████▊ | 17705/20000 [45:32<08:33,  4.47it/s]




 89%|████████▊ | 17707/20000 [45:32<07:01,  5.44it/s]




 89%|████████▊ | 17709/20000 [45:32<07:47,  4.90it/s]




 89%|████████▊ | 17710/20000 [45:33<09:27,  4.04it/s]




 89%|████████▊ | 17711/20000 [45:33<08:40,  4.40it/s]




 89%|████████▊ | 

In [ ]:
df_recipe_dataset_path = os.path.join(CACHE_DIR, 'ingr_cleaned_recipes.pkl')
df_recipes.to_pickle(df_recipe_dataset_path)

In [ ]:
ingr_proc_dataset_path = os.path.join(CACHE_DIR, 'ingr_proc_20210512.pkl')
with open(ingr_proc_dataset_path, "wb") as ingr_proc_fn:
  pickle.dump(IngrProc, ingr_proc_fn)

# Unpickling results from preprocessing

In [ ]:
df_recipe_dataset_path = os.path.join(CACHE_DIR, 'ingr_cleaned_recipes.pkl')
df_recipes = pd.read_pickle(df_recipe_dataset_path)

ingr_proc_dataset_path = os.path.join(CACHE_DIR, 'ingr_proc_20210512.pkl')
with open(ingr_proc_dataset_path, "rb") as ingr_proc_fn:
  IngrProc = pickle.load(ingr_proc_fn)

In [ ]:
df_recipes

,recipe,title,ingredients,instructions
0,🍴 Slow Cooker Chicken and Dumplings\n\n🥑\n• 4 ...,Slow Cooker Chicken and Dumplings,"[skinless boneless chicken breast halves, butt...","[Place the chicken, butter, soup, and onion in..."
1,🍴 Awesome Slow Cooker Pot Roast\n\n🥑\n• 2 (10....,Awesome Slow Cooker Pot Roast,"[condensed cream, mushroom soup, soup mix, wat...","[In a slow cooker, mix cream of mushroom soup,..."
2,🍴 Brown Sugar Meatloaf\n\n🥑\n• 1/2 cup packed ...,Brown Sugar Meatloaf,"[sugar, ketchup, beef, milk, eggs, salt, black...",[Preheat oven to 350 degrees F (175 degrees C)...
3,🍴 Best Chocolate Chip Cookies\n\n🥑\n• 1 cup bu...,Best Chocolate Chip Cookies,"[butter, sugar, sugar, eggs, vanilla extract, ...",[Preheat oven to 350 degrees F (175 degrees C)...
4,🍴 Homemade Mac and Cheese Casserole\n\n🥑\n• 8 ...,Homemade Mac and Cheese Casserole,"[pasta, florets, onion, garlic, butter, flour,...",[Preheat oven to 350 degrees F. Line a 2-quart...
...,...,...,...,...
20159,🍴 Mediterranean Made Rights (Loose Meat Sandwi...,Mediterranean Made Rights (Loose Meat Sandwiches),"[olive oil, lamb, onion, lemon zest, oregano, ...",[Heat the olive oil in a large skillet over me...
20160,🍴 Pfefferkuchen\n\n🥑\n• 1 1/2 cups molasses\n•...,Pfefferkuchen,"[molasses, honey, sugar, seed, cardamom, ginge...","[Bring molasses, honey, and sugar to a simmer ..."
20161,🍴 Georgia's Tennessee Jam Cake\n\n🥑\n• 1 cup b...,Georgia's Tennessee Jam Cake,"[butter, sugar, eggs, baking soda, water, seed...",[Preheat the oven to 350 degrees F (175 degree...
20162,🍴 Poached Eggs and Asparagus\n\n🥑\n• 4 eggs\n•...,Poached Eggs and Asparagus,"[eggs, bouillon, asparagus, bread, cheese, but...",[Fill a saucepan half way full of water. Bring...


# Train a network which will generate just the ingredient list.

Or rather, [title + ingredient list] from title.

In [ ]:
def get_recipe_and_ingredients(recipe):
  recipe_and_ingredients, _ = re.split("🥣", recipe)
  return recipe_and_ingredients

In [ ]:
recipes[0]

'🍴 Slow Cooker Chicken and Dumplings\n\n🥑\n• 4 skinless, boneless chicken breast halves\n• 2 tablespoons butter\n• 2 (10.75 ounce) cans condensed cream of chicken soup\n• 1 onion, finely diced\n• 2 (10 ounce) packages refrigerated biscuit dough, torn into pieces\n\n🥣\n‣ Place the chicken, butter, soup, and onion in a slow cooker, and fill with enough water to cover.\n‣ Cover, and cook for 5 to 6 hours on High. About 30 minutes before serving, place the torn biscuit dough in the slow cooker. Cook until the dough is no longer raw in the center.'

In [ ]:
get_recipe_and_ingredients(recipes[0])

'🍴 Slow Cooker Chicken and Dumplings\n\n🥑\n• 4 skinless, boneless chicken breast halves\n• 2 tablespoons butter\n• 2 (10.75 ounce) cans condensed cream of chicken soup\n• 1 onion, finely diced\n• 2 (10 ounce) packages refrigerated biscuit dough, torn into pieces\n\n'

In [ ]:
df_recipes['recipe+ingr'] = df_recipes['recipe'].apply(lambda x: get_recipe_and_ingredients(x))

In [ ]:
# # Make this tractable? It keeps crashing with 20000

# df_recipes = df_recipes[:200]
# df_recipes['recipe+ingr']

0      🍴 Slow Cooker Chicken and Dumplings\n\n🥑\n• 4 ...
1      🍴 Awesome Slow Cooker Pot Roast\n\n🥑\n• 2 (10....
2      🍴 Brown Sugar Meatloaf\n\n🥑\n• 1/2 cup packed ...
3      🍴 Best Chocolate Chip Cookies\n\n🥑\n• 1 cup bu...
4      🍴 Homemade Mac and Cheese Casserole\n\n🥑\n• 8 ...
                             ...                        
197    🍴 Slow Cooker Buffalo Chicken Sandwiches\n\n🥑\...
198    🍴 Chicken Milano\n\n🥑\n• 1 tablespoon butter\n...
199    🍴 Classic Waffles\n\n🥑\n• 2 cups all-purpose f...
200    🍴 Cheesy Ham and Hash Brown Casserole\n\n🥑\n• ...
201    🍴 Grandma Johnson's Scones\n\n🥑\n• 1 cup sour ...
Name: recipe+ingr, Length: 200, dtype: object

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer(char_level=False, lower=True, split=' ', oov_token='<UNK>')
tokenizer.fit_on_texts(df_recipes['recipe+ingr'])

In [ ]:
encoded_recipe_ingrs = tokenizer.texts_to_sequences(recipes) # recipes[:200]
encoded_recipe_ingrs[0]

[8,
 292,
 298,
 28,
 16,
 782,
 9,
 2,
 6,
 113,
 91,
 28,
 115,
 131,
 2,
 4,
 13,
 22,
 2,
 4,
 70,
 149,
 12,
 117,
 138,
 36,
 130,
 28,
 81,
 2,
 3,
 30,
 119,
 50,
 2,
 4,
 70,
 12,
 191,
 427,
 671,
 425,
 418,
 59,
 126,
 1,
 1,
 3480,
 322,
 28,
 22,
 81,
 16,
 30,
 175,
 296,
 292,
 298,
 16,
 1,
 79,
 2478,
 44,
 25,
 911,
 1,
 911,
 16,
 1626,
 125,
 54,
 25,
 48,
 4520,
 614,
 1305,
 1283,
 867,
 1501,
 2271,
 1077,
 3480,
 322,
 418,
 671,
 425,
 175,
 322,
 292,
 298,
 1626,
 1632,
 322,
 425,
 2662,
 457,
 1,
 598,
 175,
 322,
 996]

In [ ]:
recipe_ingr_maxlen = max([len(r) for r in encoded_recipe_ingrs])
recipe_ingr_maxlen

424

In [ ]:
padded_recipe_ingrs = pad_sequences(encoded_recipe_ingrs,
                                    padding='post',
                                    truncating='post',
                                    maxlen=recipe_ingr_maxlen+1)  # Guarantee at least 1 padding word at end

In [ ]:
padded_recipe_ingrs

array([[   8,  292,  298, ...,    0,    0,    0],
       [   8,  878,  292, ...,    0,    0,    0],
       [   8,   65,   19, ...,    0,    0,    0],
       ...,
       [   8, 7289, 3083, ...,    0,    0,    0],
       [   8, 1560,   49, ...,    0,    0,    0],
       [   8, 7290, 1938, ...,    0,    0,    0]], dtype=int32)

In [ ]:
vocab_size = len(tokenizer.word_counts) + 2
vocab_size

7292

In [ ]:
idx2word = tokenizer.sequences_to_texts([[idx] for idx in range(vocab_size)])
idx2word

['<UNK>',
 '<UNK>',
 '•',
 '1',
 '2',
 'cup',
 '4',
 'teaspoon',
 '🍴',
 '🥑',
 '3',
 'cups',
 'ounce',
 'tablespoons',
 'chopped',
 'salt',
 'and',
 'pepper',
 'ground',
 'sugar',
 'tablespoon',
 'white',
 'butter',
 'cheese',
 'oil',
 'to',
 'teaspoons',
 'garlic',
 'chicken',
 'taste',
 'onion',
 'fresh',
 'can',
 'flour',
 '8',
 'black',
 'cream',
 'powder',
 'all',
 'purpose',
 'sauce',
 'sliced',
 'package',
 'pound',
 'water',
 'minced',
 'milk',
 'dried',
 '6',
 'eggs',
 'diced',
 'baking',
 'green',
 'olive',
 '5',
 'vanilla',
 'or',
 'extract',
 'red',
 'into',
 'shredded',
 'cut',
 'cloves',
 'vegetable',
 'juice',
 'brown',
 'lemon',
 'drained',
 'egg',
 'peeled',
 '10',
 'large',
 'beef',
 'inch',
 'chocolate',
 'pounds',
 'bread',
 'ounces',
 'cinnamon',
 'with',
 'dry',
 'soup',
 'grated',
 'tomatoes',
 'softened',
 'vinegar',
 'bell',
 'mix',
 'crushed',
 'pinch',
 '16',
 'boneless',
 'soda',
 'optional',
 'tomato',
 '12',
 'potatoes',
 'frozen',
 '15',
 'beans',
 'rice',

In [ ]:
idx2word[292]

'slow'

In [ ]:
print(vocab_size)
print(len(padded_recipe_ingrs))

7292
20000


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(padded_recipe_ingrs)
dataset

<TensorSliceDataset shapes: (425,), types: tf.int32>

In [ ]:
def split_input_target(recipe):
  input_text = recipe[:-1]
  target_text = recipe[1:]
  return input_text, target_text

dataset = dataset.map(split_input_target)
dataset

<MapDataset shapes: ((424,), (424,)), types: (tf.int32, tf.int32)>

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 1000

# Create training batches
dataset = dataset \
  .shuffle(BUFFER_SIZE) \
  .batch(BATCH_SIZE, drop_remainder=True) \
  .repeat()

dataset

<RepeatDataset shapes: ((64, 424), (64, 424)), types: (tf.int32, tf.int32)>

## Build the model

In [ ]:
def build_model(vocab_size, embed_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(
            rnn_units,
            return_sequences=True,
            stateful=True,
            recurrent_initializer='glorot_uniform'
        ),
        tf.keras.layers.Dense(vocab_size)
    ])

    return model

In [ ]:
EMBED_DIM = 256
RNN_UNITS = 512
MODEL_NAME = "rnn_word_ingr_20000_7"

In [ ]:
model = build_model(
    vocab_size=vocab_size,
    embed_dim=EMBED_DIM,
    rnn_units=RNN_UNITS,
    batch_size=BATCH_SIZE)

model.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (64, None, 256)           1866752   
_________________________________________________________________
gru_27 (GRU)                 (64, None, 512)           1182720   
_________________________________________________________________
dense_27 (Dense)             (64, None, 7292)          3740796   
Total params: 6,790,268
Trainable params: 6,790,268
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape) # (batch_size, sequence_length, vocab_size)

(64, 424, 7292)


In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
    
example_batch_loss = loss(target_example_batch, example_batch_predictions)
prediction_shape = example_batch_predictions.shape
scalar_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", prediction_shape)
print("scalar_loss:      ", scalar_loss)
print("exp(scalar_loss): ", np.exp(scalar_loss))
print("vocab size      : ", vocab_size)
print("If all went right, exp(scalar loss) should be approximately equal to vocab size")

Prediction shape:  (64, 424, 7292)
scalar_loss:       7.7580867
exp(scalar_loss):  2340.4224
vocab size      :  7292
If all went right, exp(scalar loss) should be approximately equal to vocab size


In [ ]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    patience=5,
    monitor='loss',
    restore_best_weights=True,
    verbose=1
)

In [ ]:
# Create a checkpoints directory.
checkpoint_dir = os.path.join(CACHE_DIR, MODEL_NAME)
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'ckpt_{epoch}'),
    save_weights_only=True
)

In [ ]:
def restore_checkpoint(model):
    latest_checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)

    if not latest_checkpoint_path:
        print('Checkpoint not found')
        return model, 0

    print("Checkpoint found")
    print('Path:', latest_checkpoint_path)

    model.load_weights(latest_checkpoint_path)
    # model = load_model(latest_checkpoint_path)

    latest_checkpoint_name = os.path.split(latest_checkpoint_path)[-1]
    print('Name:', latest_checkpoint_name)

    latest_epoch = latest_checkpoint_name.split('_')[-1]
    print('Epoch:', latest_epoch)

    return model, int(latest_epoch)

In [ ]:
TOTAL_EPOCHS = 10
STEPS_PER_EPOCH = 1500

model, initial_epoch = restore_checkpoint(model)

Checkpoint not found


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=loss
)

Train model

In [ ]:
history = model.fit(
    x=dataset,
    epochs=TOTAL_EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    initial_epoch=initial_epoch,
    verbose=True,
    callbacks=[
        checkpoint_callback,
        early_stopping_callback
    ]
)

# Saving the trained model to file (to be able to re-use it later).
model_name = os.path.join(CACHE_DIR, MODEL_NAME, f'{MODEL_NAME}.h5')
model.save(model_name, save_format='h5')
# model.save(model_name)

Epoch 1/10
1500/1500 [==============================] - 319s 212ms/step - loss: 2.0474
Epoch 2/10
1500/1500 [==============================] - 316s 211ms/step - loss: 1.1111
Epoch 3/10
1500/1500 [==============================] - 316s 211ms/step - loss: 0.9511
Epoch 4/10
1500/1500 [==============================] - 315s 210ms/step - loss: 0.7978
Epoch 5/10
1500/1500 [==============================] - 316s 211ms/step - loss: 0.7007
Epoch 6/10
1500/1500 [==============================] - 316s 211ms/step - loss: 0.6405
Epoch 7/10
1500/1500 [==============================] - 315s 210ms/step - loss: 0.6080
Epoch 8/10
1500/1500 [==============================] - 317s 211ms/step - loss: 0.5799
Epoch 9/10
1500/1500 [==============================] - 316s 210ms/step - loss: 0.5565
Epoch 10/10
1500/1500 [==============================] - 317s 211ms/step - loss: 0.5375


## Predictions

In [ ]:
def generate_recipe(model, start_string, num_generate=1000, temperature=0.8):
    TITLE_START = "🍴 "
    # Evaluation step (generating text using the learned model)

    padded_start_string = TITLE_START + start_string

    # Converting our start string to numbers (vectorizing).
    input_eval = np.array(tokenizer.texts_to_sequences([padded_start_string]))

    # Empty string to store our results.
    text_generated = []

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        if predicted_id == 0: # stop if we start generating the padding token
            break

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2word[predicted_id])

    return (start_string + ' '.join(text_generated))

In [ ]:
# Restore latest checkpoint and change batch to 1
model = build_model(vocab_size, EMBED_DIM, RNN_UNITS, batch_size=1)
model, _ = restore_checkpoint(model)
model.build(tf.TensorShape([1, None]))

Checkpoint found
Path: ./drive/Shared drives/Capstone/tmp/rnn_word_ingr_20000_7/ckpt_10
Name: ckpt_10
Epoch: 10


In [ ]:
generated_text = generate_recipe(model, start_string="Pork ", num_generate=500, temperature=0.7)
print(generated_text)

Pork processor know fridge cannelloni know fridge fridge skirt york york york york know fridge know know york york york york york york york york york york know shelf lunchroom know kick fridge know korma know kirsch colors dente know fridge know latkes kick know know dente garam circular dente know segments kick c cheesesteak circular dente know carbonara know know yoghurt fondant colors ounce know kirsch c know kirsch cacao know kick unpopped unflavored morton® thermostat kick york york unflavored morton® ople's know faced unflavored morton® circular blistered dollar know unflavored vidalia® unflavored vidalia® unflavored vidalia® unflavored vidalia® unflavored cob purpose bisquick ounce softly unflavored percent plantain mesquite york lunchroom know angle kick lunchroom unflavored morton® blistered mop percent aren't know percent know breakstone's confectioners dente know dente know kirsch dente shaken know know c confectioners know know know c faced c faced know fridge dente dente d